#### This notebook demonstrates the use of the fairness adjuster, using the same structure as the AIF360 adversarial debiasing example

The source notebook can be found here:
https://github.com/Trusted-AI/AIF360/blob/main/examples/demo_adversarial_debiasing.ipynb

In [1]:
%matplotlib inline
# Load all necessary packages
import sys

sys.path.append("../")
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import (
    load_preproc_data_adult,
    load_preproc_data_compas,
    load_preproc_data_german,
)

from aif360.algorithms.inprocessing.fairness_adjuster import FairnessAdjuster

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf

import numpy as np
import pandas as pd

tf.disable_eager_execution()

2024-12-09 19:59:39.547270: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-09 19:59:39.549863: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-09 19:59:39.578970: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-09 19:59:39.578995: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-09 19:59:39.579018: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
NUM_EPOCHS = 50
SEED = 1

np.random.seed(SEED)

#### Load dataset and set options

In [3]:
# Get the dataset and split into train and test
dataset_orig = load_preproc_data_adult()

privileged_groups = [{"sex": 1}]
unprivileged_groups = [{"sex": 0}]

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

In [4]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(
    dataset_orig_train.privileged_protected_attributes,
    dataset_orig_train.unprivileged_protected_attributes,
)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(34189, 18)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['sex', 'race']


#### Privileged and unprivileged protected attribute values

[array([1.]), array([1.])] [array([0.]), array([0.])]


#### Dataset feature names

['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (decade)=50', 'Age (decade)=60', 'Age (decade)=>=70', 'Education Years=6', 'Education Years=7', 'Education Years=8', 'Education Years=9', 'Education Years=10', 'Education Years=11', 'Education Years=12', 'Education Years=<6', 'Education Years=>12']


#### Metric for original training data

In [5]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(
    dataset_orig_train,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)
display(Markdown("#### Original training dataset"))
print(
    "Train set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_orig_train.mean_difference()
)
metric_orig_test = BinaryLabelDatasetMetric(
    dataset_orig_test,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)
print(
    "Test set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_orig_test.mean_difference()
)

#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.190244
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.204482


In [6]:
min_max_scaler = MaxAbsScaler()
dataset_orig_train.features = min_max_scaler.fit_transform(dataset_orig_train.features)
dataset_orig_test.features = min_max_scaler.transform(dataset_orig_test.features)
metric_scaled_train = BinaryLabelDatasetMetric(
    dataset_orig_train,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)
display(
    Markdown(
        "#### Scaled dataset - Verify that the scaling does not affect the group label statistics"
    )
)
print(
    "Train set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_scaled_train.mean_difference()
)
metric_scaled_test = BinaryLabelDatasetMetric(
    dataset_orig_test,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)
print(
    "Test set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_scaled_test.mean_difference()
)

#### Scaled dataset - Verify that the scaling does not affect the group label statistics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.190244
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.204482


### Learn plan classifier without debiasing

In [7]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
sess = tf.Session()
plain_model = FairnessAdjuster(
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups,
    scope_name="plain_classifier",
    debias=False,
    sess=sess,
    num_epochs=NUM_EPOCHS,
    seed=SEED,
)

In [8]:
plain_model.fit(dataset_orig_train)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2024-12-09 19:59:42.955843: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled


epoch 0; iter: 0; batch classifier loss: 0.777370
epoch 0; iter: 200; batch classifier loss: 0.434442
epoch 1; iter: 0; batch classifier loss: 0.404269
epoch 1; iter: 200; batch classifier loss: 0.326910
epoch 2; iter: 0; batch classifier loss: 0.350508
epoch 2; iter: 200; batch classifier loss: 0.493786
epoch 3; iter: 0; batch classifier loss: 0.519997
epoch 3; iter: 200; batch classifier loss: 0.501568
epoch 4; iter: 0; batch classifier loss: 0.467276
epoch 4; iter: 200; batch classifier loss: 0.445991
epoch 5; iter: 0; batch classifier loss: 0.375114
epoch 5; iter: 200; batch classifier loss: 0.497320
epoch 6; iter: 0; batch classifier loss: 0.462191
epoch 6; iter: 200; batch classifier loss: 0.418625
epoch 7; iter: 0; batch classifier loss: 0.434002
epoch 7; iter: 200; batch classifier loss: 0.392433
epoch 8; iter: 0; batch classifier loss: 0.401344
epoch 8; iter: 200; batch classifier loss: 0.536455
epoch 9; iter: 0; batch classifier loss: 0.369766
epoch 9; iter: 200; batch classi

In [9]:
# Apply the plain model to test data
dataset_nodebiasing_train = plain_model.predict(dataset_orig_train)
dataset_nodebiasing_test = plain_model.predict(dataset_orig_test)

In [10]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(
    dataset_nodebiasing_train,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)

print(
    "Train set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_dataset_nodebiasing_train.mean_difference()
)

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(
    dataset_nodebiasing_test,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)

print(
    "Test set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_dataset_nodebiasing_test.mean_difference()
)

display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(
    dataset_orig_test,
    dataset_nodebiasing_test,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)
print(
    "Test set: Classification accuracy = %f"
    % classified_metric_nodebiasing_test.accuracy()
)
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5 * (TPR + TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print(
    "Test set: Disparate impact = %f"
    % classified_metric_nodebiasing_test.disparate_impact()
)
print(
    "Test set: Equal opportunity difference = %f"
    % classified_metric_nodebiasing_test.equal_opportunity_difference()
)
print(
    "Test set: Average odds difference = %f"
    % classified_metric_nodebiasing_test.average_odds_difference()
)
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.233568
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.239190


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.802703
Test set: Balanced classification accuracy = 0.673437
Test set: Disparate impact = 0.000000
Test set: Equal opportunity difference = -0.494238
Test set: Average odds difference = -0.309304
Test set: Theil_index = 0.172950


### Apply in-processing algorithm based on adversarial learning

In [11]:
tf.reset_default_graph()
sess2 = tf.Session()

In [12]:
# Learn parameters with debias set to True
debiased_model = FairnessAdjuster(
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups,
    scope_name="debiased_classifier",
    adversary_loss_weight=0.1,
    debias=True,
    sess=sess2,
    num_epochs=NUM_EPOCHS,
    seed=SEED,
)

In [13]:
debiased_model.fit(dataset_orig_train)

epoch 0; iter: 0; batch classifier loss: 0.777370
epoch 0; iter: 200; batch classifier loss: 0.434442
epoch 1; iter: 0; batch classifier loss: 0.404269
epoch 1; iter: 200; batch classifier loss: 0.326910
epoch 2; iter: 0; batch classifier loss: 0.350508
epoch 2; iter: 200; batch classifier loss: 0.493786
epoch 3; iter: 0; batch classifier loss: 0.519997
epoch 3; iter: 200; batch classifier loss: 0.501568
epoch 4; iter: 0; batch classifier loss: 0.467276
epoch 4; iter: 200; batch classifier loss: 0.445991
epoch 5; iter: 0; batch classifier loss: 0.375114
epoch 5; iter: 200; batch classifier loss: 0.497320
epoch 6; iter: 0; batch classifier loss: 0.462191
epoch 6; iter: 200; batch classifier loss: 0.418625
epoch 7; iter: 0; batch classifier loss: 0.434002
epoch 7; iter: 200; batch classifier loss: 0.392433
epoch 8; iter: 0; batch classifier loss: 0.401344
epoch 8; iter: 200; batch classifier loss: 0.536455
epoch 9; iter: 0; batch classifier loss: 0.369766
epoch 9; iter: 200; batch classi

In [14]:
# Apply the plain model to test data
dataset_debiasing_train = debiased_model.predict(dataset_orig_train)
dataset_debiasing_test = debiased_model.predict(dataset_orig_test)

2024-12-09 20:00:18.397390: W tensorflow/c/c_api.cc:305] Operation '{name:'debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Assign' id:268 op device:{requested: '', assigned: ''} def:{{{node debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1, debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [15]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
print(
    "Train set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_dataset_nodebiasing_train.mean_difference()
)
print(
    "Test set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_dataset_nodebiasing_test.mean_difference()
)

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(
    dataset_debiasing_train,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)

print(
    "Train set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_dataset_debiasing_train.mean_difference()
)

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(
    dataset_debiasing_test,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)

print(
    "Test set: Difference in mean outcomes between unprivileged and privileged groups = %f"
    % metric_dataset_debiasing_test.mean_difference()
)


display(Markdown("#### Plain model - without debiasing - classification metrics"))
print(
    "Test set: Classification accuracy = %f"
    % classified_metric_nodebiasing_test.accuracy()
)
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5 * (TPR + TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print(
    "Test set: Disparate impact = %f"
    % classified_metric_nodebiasing_test.disparate_impact()
)
print(
    "Test set: Equal opportunity difference = %f"
    % classified_metric_nodebiasing_test.equal_opportunity_difference()
)
print(
    "Test set: Average odds difference = %f"
    % classified_metric_nodebiasing_test.average_odds_difference()
)
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())


display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(
    dataset_orig_test,
    dataset_debiasing_test,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)
print(
    "Test set: Classification accuracy = %f"
    % classified_metric_debiasing_test.accuracy()
)
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5 * (TPR + TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print(
    "Test set: Disparate impact = %f"
    % classified_metric_debiasing_test.disparate_impact()
)
print(
    "Test set: Equal opportunity difference = %f"
    % classified_metric_debiasing_test.equal_opportunity_difference()
)
print(
    "Test set: Average odds difference = %f"
    % classified_metric_debiasing_test.average_odds_difference()
)
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.233568
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.239190


#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.064935
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.069512


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.802703
Test set: Balanced classification accuracy = 0.673437
Test set: Disparate impact = 0.000000
Test set: Equal opportunity difference = -0.494238
Test set: Average odds difference = -0.309304
Test set: Theil_index = 0.172950


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.786460
Test set: Balanced classification accuracy = 0.669223
Test set: Disparate impact = 0.666324
Test set: Equal opportunity difference = -0.008763
Test set: Average odds difference = -0.003839
Test set: Theil_index = 0.173868



    References:
    [1] B. H. Zhang, B. Lemoine, and M. Mitchell, "Mitigating UnwantedBiases with Adversarial Learning," 
    AAAI/ACM Conference on Artificial Intelligence, Ethics, and Society, 2018.

Print predicted labels for the test set.

In [17]:
pd.Series(dataset_debiasing_test.labels.reshape(-1)).value_counts()

0.0    11939
1.0     2714
Name: count, dtype: int64

In [18]:
pd.Series(dataset_nodebiasing_test.labels.reshape(-1)).value_counts()

0.0    12313
1.0     2340
Name: count, dtype: int64